# Hass Client

Create a long lived access token https://www.home-assistant.io/docs/authentication/

In [18]:
import yaml

secrets = yaml.load(open("secrets.yaml"), Loader=yaml.Loader)
api_token = secrets["hass_token"]
hostport = secrets["hostport"]

In [19]:

import hass_client

url = f"wss://{hostport}/api/websocket"

client = hass_client.HomeAssistantClient(url, api_token)
await client.connect()

In [20]:
states = await client.get_states()
len(states)

683